# This is a introducation for paired trades analysis program
Alan, 2019-09-13

Introduction:  
This script generates paired trades information  
Inputs:  
    * Assuming all data record are sorted with "Time"  
    * market data file(csv)  
    * trades file(csv)  
Output:  
    * paired trades file(csv)  
NOTE:  
    * This implementation used two dynamicaly maintained  
    * user defined structure:  
    **    position   = stored all live positions information  
    **    cur_market = stored current live market information given time  
    * Program structure is simple by only three functions and one loop  
    * Include liquidity measure as "GOOD" if trade price aline with market 
    * "BAD" if trade price off the market
    * The momery costs is O(n), since dynamic sturcture is small  
    *    only for reading all raw data costs  
    * Total program running is O(n) we only go through the raw  
    * data file once!  
author:  Shaolun Du  
contact: Shaolun.du@gmail.com  

In [7]:
###---Importing support libs---###
import csv
from collections import deque

In [8]:
###---User defined functions---###
def market_update( market, 
                   cur_time,
                   cur_market ):
    """ Return currnet market information
        given trading time in "trade"
    """
    if len(market) == 0:
        # No more new market information
        # return latest market
        return cur_market
    if len(cur_market) == 0:
        # Initilize market information
        row = market.popleft()
        cur_market[row[1]] = [int(row[0]),float(row[2]),float(row[3])]
        cur_time = int(row[0])
        while cur_time >= int(market[0][0]):
            row = market.popleft()
            cur_market[row[1]] = [int(row[0]),float(row[2]),float(row[3])]
        return cur_market
    while cur_time >= int(market[0][0]):
        row = market.popleft()
        cur_market[row[1]] = [int(row[0]),float(row[2]),float(row[3])]
        if len(market) == 0:
            # No more new market information
            # return latest market
            return cur_market
    return cur_market

In [9]:
def generate_record( symbol, quant,
                     open_time, open_side, open_price,liq_1,
                     close_time, close_side, close_price,liq_2 ):
    # Generate trading record with requirement
    if close_side == "S":
        multi = 1
    else:
        multi = -1
    profit = multi*quant*(close_price-open_price)
    record = [symbol, quant,
              open_time, open_side, open_price,liq_1,
              close_time, close_side, close_price,liq_2,profit]
    return record

In [10]:
def execution( cur_market,
               position,
               trade ):
    """ Return trade execution information 
        and positions after trading
        given current positions and current market 
    """
    if len(trade) == 0:
        raise Exception("ERROR in execution: No trades information.")
    record = []
    time   = int(trade[0])
    symbol = trade[1]
    side   = trade[2]
    price  = float(trade[3])
    quant  = int(trade[4])
    market = cur_market[symbol]
    if side == "B" and price > market[2]:
        liquidity = "BAD"
    elif side == "S" and price < market[1]:
        liquidity = "BAD"
    else:
        liquidity = "GOOD"
    """New trade or current no positions
    """
    if symbol not in position.keys() or len(position[symbol]) == 0:
        position[symbol] = deque()
        position[symbol].append([time,side,price,quant,market[1],market[2],liquidity])
        return position, record
    """ Pair this trade with current positions
        need to check cases:
            1.offset to only one or several trades
            2.any trades left over after offset
            3.if some new trade quant left over 
                need to treat them as neww open trades
        Return record list may be more than one records!
    """
    cur_pos = position[symbol] # Get current positions
    while quant > 0:
        if len(cur_pos) == 0:
            cur_pos.append([time,side,price,quant,market[1],market[2],liquidity])
            break
        cur_hold = cur_pos[0]
        if cur_hold[1] == side:
            # Same direction with current holding
            cur_pos.append([time,side,price,quant,market[1],market[2],liquidity])
            quant = 0
        else:
            # Offset direction to current holding
            # Find a "Pair" here!
            if cur_hold[3] == quant:
                # Exact matching
                recs = generate_record(symbol, quant,
                                       cur_hold[0], cur_hold[1], cur_hold[2],cur_hold[6],
                                       time, side, price,liquidity)
                record.append(recs)
                cur_pos.popleft()
                quant = 0
            elif cur_hold[3] > quant:
                # Offset part of current holding
                cur_hold[3] -= quant
                recs = generate_record(symbol, quant,
                                       cur_hold[0], cur_hold[1], cur_hold[2],cur_hold[6],
                                       time, side, price,liquidity)
                record.append(recs)
                quant = 0
            else:
                # Over sell/buy to current holding 
                # Will open new opposite durection
                # after this order
                quant -= cur_hold[3]
                recs = generate_record(symbol, cur_hold[3],
                                       cur_hold[0], cur_hold[1], cur_hold[2],cur_hold[6],
                                       time, side, price,liquidity)
                record.append(recs)
                cur_pos.popleft()
    return position,record

In [11]:
###---Main function entry here!
###---Loading in market and trades data
trades_f = open('Trades.txt', newline='')
market_f = open('Quotes.txt', newline='')
trades   = list(csv.reader(trades_f, delimiter=',', quotechar='|'))[1:]
market   = deque(csv.reader(market_f, delimiter=',', quotechar='|'))
market.popleft() # Pop out the first line
###---Loop through trades and dynamicly update market
###---information for each trade records
###---dynamicly maintains positions inforamtion as well
paired_trade = [] # Answer dictionary
cur_market   = {} # Current market trading information
position     = {} # Current holiding positions
record       = [] # Paired trades record
cur_market   = market_update(market,0,cur_market) # Initilize market
for t in trades:
    cur_time        = int(t[0])
    cur_market      = market_update( market, cur_time, cur_market ) # market information update
    position,record = execution( cur_market, position, t ) # trade execution
    if len(record) > 0:
        # Collecting paired trades
        for ele in record:
            paired_trade.append( ele )

In [12]:
###---Start outputing results
ans_string = ""
for rec in paired_trade:
    for ele in rec:
        ans_string += str(ele)+","
    ans_string += "\n"
output_file = open("paired_trades.txt","w") 
output_file.write(ans_string)
output_file.close()

In [13]:
print(paired_trade)

[['DEF', 100, 2, 'S', 41.1, 'GOOD', 3, 'B', 16.57, 'GOOD', 2453.0], ['ABC', 100, 1, 'B', 23.45, 'BAD', 6, 'S', 24.69, 'BAD', 124.0000000000002], ['ABC', 200, 6, 'S', 24.69, 'BAD', 6, 'B', 18.06, 'GOOD', 1326.0000000000005], ['DEF', 100, 2, 'S', 41.1, 'GOOD', 9, 'B', 43.66, 'BAD', -255.99999999999952], ['DEF', 200, 3, 'S', 43.15, 'GOOD', 9, 'B', 43.66, 'BAD', -101.9999999999996]]
